## Imports

In [ ]:
import sys

sys.path.append("..")

In [ ]:
from src.transformers import *
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

## Load Data

In [ ]:
# Set default logging level
# Change to logging.INFO to see related output
logging.basicConfig(level=logging.INFO, force=True)

In [ ]:
# Read data and set indices
train = pd.read_csv("../data/raw/train.csv")
test = pd.read_csv("../data/raw/test.csv")

train.set_index("listing_id", inplace=True)
train = train[~train.index.duplicated(keep="first")]
train = train.reset_index()

# NOTE: Do not remove duplicates in test as kaggle expects 5000 entries in the submission file
# test.set_index("listing_id", inplace=True)
# test = test[~test.index.duplicated(keep="first")]

## Construct Pipeline

In [ ]:
data_pipeline = Pipeline(
    [
        ("pre_processing", PreProcessing()),
        ("imp_power", GroupMissingValueImputer("power", ["type_of_vehicle"], "mean")),
        (
            "imp_engine_cap",
            GroupMissingValueImputer("engine_cap", ["make", "type_of_vehicle"], "mean"),
        ),
        (
            "imp_curb_weight",
            GroupMissingValueImputer("curb_weight", ["make", "type_of_vehicle"]),
        ),
        #         ("add_cat", SplitValuesToColumn("category")),
        # Columns 9 to 15
        (
            "curb_weight",
            CarSpecificationsTransformer(
                "curb_weight", ["make", "model", "type_of_vehicle", "manufactured"]
            ),
        ),
        (
            "power",
            CarSpecificationsTransformer(
                "power", ["make", "model", "type_of_vehicle", "manufactured"]
            ),
        ),
        (
            "fuel_type",
            CarSpecificationsTransformer("fuel_type", ["make", "model"], "mode"),
        ),
        (
            "engine_cap",
            CarSpecificationsTransformer(
                "engine_cap", ["make", "model", "type_of_vehicle", "manufactured"]
            ),
        ),
        ("fuel_type_missing", CarSpecsMissingWithTypeOfVehicle(["fuel_type"], "mode")),
        (
            "car_spec_missing",
            CarSpecsMissingWithTypeOfVehicle(
                ["curb_weight", "power", "engine_cap"], "mean"
            ),
        ),
        (
            "convert_value_to_category",
            ColumnValuesToCategory(
                "engine_cap",
                "engine_cap_range",
                [0, 600, 1000, 1600, 3000, np.inf],
                [
                    "EC<=600 cc",
                    "600 cc < EC <= 1000 cc ",
                    "1000 cc < EC <= 1600 cc",
                    "1600 cc < EC <= 3000 cc",
                    "EC > 3000 cc",
                ],
            ),
        ),
        (
            "fuel_type_one_hot",
            OneHotTransformer(
                "fuel_type", ["diesel", "petrol-electric", "petrol", "electric"]
            ),
        ),
        ("transmission_one_hot", OneHotTransformer("transmission", ["auto", "manual"])),
        # Columns 15 and above
        ("coe_start_date", CoeStartDateFeatureCreator()),
        ("vehicle_age", AgeFeatureCreator()),
        (
            "omv",
            HierarchicalGroupImputer(
                "omv",
                [["make", "model", "vehicle_age"], ["make", "model"]],
                "mean",
                True,
            ),
        ),
        ("coe", CoeTransformer()),
        ("arf", ArfTransformer()),
        ("parf", ParfFeatureCreator()),
        ("coe_rebate", CoeRebateFeatureCreator()),
        ("dereg_value_computed", DeregValueComputedFeatureCreator()),
        ("dereg_value", DeregValueTransformer()),
        (
            "vehicle_age_bins",
            ColumnValuesToCategory(
                "vehicle_age",
                "vehicle_age_bins",
                [0, 10, 20, 35, 50, np.inf],
                ["0-10", "10-20", "20-35", "35-50", ">50"],
            ),
        ),
        # Ideally, depreciation should be (price - parf) / no_of_coe_years_left
        #         modified_x.loc[depreciation_mask, "depreciation"] = (
        #             X.loc[depreciation_mask, "price"] - X.loc[depreciation_mask, "parf"]
        #         ) / 10
        # But this depends on price which is unavailable for test dataset so we resort to
        # using the mean of the below hierarchies
        (
            "depreciation",
            HierarchicalGroupImputer(
                "depreciation",
                [
                    ["make", "model", "vehicle_age_bins"],
                    ["make", "vehicle_age_bins"],
                    ["vehicle_age_bins"],
                    ["make"],
                ],
                "mean",
                True,
            ),
        ),
        (
            "mileage",
            HierarchicalGroupImputer(
                "mileage",
                [["make", "model", "vehicle_age"], ["vehicle_age"]],
                "mean",
                True,
            ),
        ),
        ("opc_scheme", OpcSchemeTransformer()),
        (
            "road_tax",
            HierarchicalGroupImputer(
                "road_tax",
                [
                    ["engine_cap_range", "opc_scheme", "vehicle_age", "fuel_type"],
                    ["engine_cap_range", "opc_scheme"],
                    ["engine_cap_range"],
                ],
                "mean",
                True,
            ),
        ),
        ("lifespan_restriction", LifespanRestrictionFeatureCreator()),
        (
            "features_count",
            CountUniqueItemsFeatureCreator("features", "features_count"),
        ),
        (
            "accessories_count",
            CountUniqueItemsFeatureCreator("accessories", "accessories_count"),
        ),
        ("brand_rank", BrandRankTransformer(),),
    ]
)

In [ ]:
cleaned_train = data_pipeline.fit_transform(train)
display(cleaned_train.head())
train.shape, cleaned_train.shape

In [ ]:
cleaned_train.columns

In [ ]:
cleaned_test = data_pipeline.transform(test)
display(cleaned_test.head())
test.shape, cleaned_test.shape

In [ ]:
cleaned_test.columns

## Write to disk

In [ ]:
# cleaned_train.to_csv("../data/processed/train.csv", index=False)
# cleaned_test.to_csv("../data/processed/test.csv", index=False)

## Sample Validation

In [ ]:
import ipywidgets

idx = np.random.randint(0, len(test))

ipywidgets.HBox(
    [
        ipywidgets.HTML(f"""<pre>{test.iloc[idx]}</pre>"""),
        ipywidgets.HTML(f"""<pre>{cleaned_test.iloc[idx]}</pre>"""),
    ]
)